In [373]:
import numpy as np
from numpy import pi

In [383]:
global g, d, m, i, v15, P0, T0, w, zt, cd, m3, S, Tv, Tz, v0
g = 9.80665 #ускорение свободного падения
d = 12.7 * 10**-3 #диаметр пули
m = 48.3 * 10**-3 #масса пули
i = 1.0628 #коэффиециен формы
v15 = 820.0 #начальная скорость пули T = 15 c
P0 = 750 #начальное давление
T0 = 15 #начальная температура
w = 0.5 #влажность воздуха
zt = 1.35 * 10**-3 #коэффиециент температуры заряда
cd = 0.0423 #коэффиециент деривации
m3 = 0.1744 #коэффиециент убывания угловой скорости
S = pi*d*d/4
Tv = (T0 + 273.15)/(1 - 3*12.7/(8.0*P0) * w)

def T(y):
    global Tv
    return Tv - 0.0065*y

def p(y):
    global Tv,P0
    return 1.225 * np.power(T(y)/Tv, 4.256) * P0*288.15/(Tv*760.0)

def s_speed(y):
    print('T(y): ',T(y))
    return 340.924 * (T(y)/288.15)**(1/2.0)

def cx(x):
    pa = np.array([0.0525, -0.9476, 8.9342, -9.4610, 0.3207, 4.2980, -1.9382])
    pb = np.array([1.0000, -15.4071, 178.6690, -580.8643, 985.5873, -853.9492, 296.9213])
    pc = np.array([0.0531, 0.9449, 90.5063, 0.1639])
    return np.polyval(pa,x**2)/np.polyval(pb,x**2) + pc[0] / (1+np.exp(-(x-pc[1])*pc[2]))+pc[3]

Tz = T0
v0 = v15*(1+zt*(Tz-15))

def Fs(i,S,p,v,a):
    return i*S*p*v*v/2*cx(v/a)

In [384]:
s_speed(0)

T(y):  289.0677902339929


341.46650904200186

In [357]:
butcher = [np.array([0, 0, 0, 0, 0, 0, 0], dtype=np.longdouble), 
           np.array([4/7, 0, 0, 0, 0, 0, 0], dtype=np.longdouble), 
           np.array([115/112, -5/16, 0, 0, 0, 0, 0], dtype=np.longdouble), 
           np.array([589/630, 5/18, -16/45, 0, 0, 0, 0], dtype=np.longdouble),
           np.array([229/1200-29/6000*5**0.5, 119/240-187/1200*5**0.5, -14/75+34/375*5**0.5, -3/100*5**0.5, 0, 0, 0], dtype=np.longdouble),
           np.array([71/2400-587/12000*5**0.5, 187/480-391/2400*5**0.5, -38/75+26/375*5**0.5, 27/80-3/400*5**0.5, (1+5**0.5)/4, 0, 0], dtype=np.longdouble),
           np.array([-49/480+43/160*5**0.5, -425/96+51/32*5**0.5, 52/15-4/5*5**0.5, -27/16+3/16*5**0.5, 5/4-3/4*5**0.5, 5/2-0.5*5**0.5, 0], dtype=np.longdouble) 
          ]
a = np.array([0, 4/7, 5/7, 6/7, (5-5**0.5)/10, (5+5**0.5)/10, 1],dtype=np.longdouble)
b = np.array([1/12, 0, 0, 0, 5/12, 5/12, 1/12], dtype=np.longdouble)

In [358]:
def mult(vect,list_):
    res = np.matrix(np.zeros(vect[0].shape))
    for v,l in zip(vect,list_):
        res+= v*l
    return res
def universal_runge_kutta(f ,u0, t, butcher, a, b):
    u = np.array([u0] * len(t))
    w = [np.matrix(np.zeros(u0.shape))] * len(butcher)
    for i,ui in enumerate(u[:-1]):
        tau = t[i+1] - t[i]
        for j,wj in enumerate(w):
            w[j] = f(u[i] + tau * mult(w, butcher[j]), t[i] + tau * a[j])
        u[i+1] = u[i] + tau * mult(w,b)
    return u, t
def f(ui, x):
    global m, g, i, S
    u = ui[0].item(0)
    gamma = ui[1].item(0)
    y = ui[2].item(0)
    t = ui[3].item(0)
    v = u * np.sqrt(1+gamma**2)
    return np.matrix([
    [-Fs(i, S, p(y), v, s_speed(y)) / (m*v)],
    [-g/u**2],
    [gamma],
    [1/u]
    ])

In [361]:
def universal_rosenbrok(f, u0, alpha, t):
    h = 10**-5
    du = np.matrix(np.zeros(u0.shape),dtype = np.float64)
    u =  [u0] * len(t)
    fu = np.zeros((u0.shape[0], u0.shape[0]),dtype = np.float64)
    for i, ui in enumerate(u[ : -1]):
        tau = t[i+1] - t[i]
        for j in range(u0.shape[0]):
            for k in range(u0.shape[0]):
                du = np.matrix( np.zeros(u0.shape),dtype = np.float64)
                du[j] = h
                val = (f(u[i]+du,t[i]) - f(u[i]-du,t[i]))/(2*h)
                fu[j][k] = val[k]
        w = np.linalg.solve( (np.eye(fu.shape[0]) - alpha * tau * fu.T), f(u[i],t[i] + tau/2))
        u[i+1] = u[i] + tau*np.real(w)
    return u, t
theta = 33.93438*0.06 *pi/180
u0 = np.array([          
    [v0*np.cos(theta)], #u
    [np.tan(theta)],   #gamma
    [0.0],                 #y
    [0.0]                  #t
    ])
ans = universal_rosenbrok(f, u0, (1.0+1j)/2.0, np.linspace(0,2000,16))
ans

([array([[8.19481197e+02],
         [3.55509656e-02],
         [0.00000000e+00],
         [0.00000000e+00]]), matrix([[7.66858664e+02],
          [3.34764160e-02],
          [4.60999743e+00],
          [1.68030931e-01]]), matrix([[7.16272839e+02],
          [3.11034492e-02],
          [8.92491050e+00],
          [3.47746103e-01]]), matrix([[6.67619417e+02],
          [2.83784217e-02],
          [1.29016892e+01],
          [5.40337829e-01]]), matrix([[6.20799859e+02],
          [2.52352082e-02],
          [1.64893880e+01],
          [7.47187336e-01]]), matrix([[5.75739202e+02],
          [2.15915277e-02],
          [1.96272792e+01],
          [9.69905393e-01]]), matrix([[5.32424433e+02],
          [1.73443472e-02],
          [2.22424043e+01],
          [1.21037255e+00]]), matrix([[4.90961708e+02],
          [1.23648481e-02],
          [2.42464498e+01],
          [1.47075987e+00]]), matrix([[4.51616695e+02],
          [6.49443386e-03],
          [2.55319736e+01],
          [1.75349647e+0

In [362]:
print('T0 = 15\n')
theta0 = 0.0
theta1 = 0.1
eps = 10**-15
n = 0 
while True:
    print("iteration",n)
    theta = (theta0+theta1)/2
    u0 = np.array([          
        [v0*np.cos(theta)], #u
        [np.tan(theta)],   #gamma
        [0.0],                 #y
        [0.0]                  #t
        ])
    #ans = universal_runge_kutta(f, u0, np.linspace(0,2000,16), butcher, a, b)[0][-1][2].item(0) 
    ans = universal_rosenbrok(f, u0, (1.0+1j)/2.0, np.linspace(0,2000,16))[0][-1][2].item(0) 
    if abs(ans) < 10**-5:
        break
    if ans > 0:
        theta1 = theta
    else:
        theta0 = theta
    n+=1
theta*180/pi/0.06

T0 = 15

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22


33.732175568401644

In [344]:
print('T0 = 15\n')
theta0 = 0.01
theta1 = 0.03
eps = 10**-15
n = 0 
while abs(theta1 - theta0) > eps:
    print("iteration",n)
    #print(abs(theta1 - theta0))
    u0 = np.array([          
        [v0*np.cos(theta1)], #u
        [np.tan(theta1)],   #gamma
        [0.0],                 #y
        [0.0]                  #t
        ])
    ans1 = universal_runge_kutta(f, u0, np.linspace(0,2000,16), butcher, a, b)
    
    u0 = np.array([          
        [v0*np.cos(theta0)],   #u
        [np.tan(theta0)],      #gamma
        [0.0],                 #y
        [0.0]                  #t
        ])
    ans0 = universal_runge_kutta(f, u0, np.linspace(0,2000,16), butcher, a, b)
    new_theta = theta1 - (theta1 - theta0) * ans1[0][-1][2].item(0) / (ans1[0][-1][2].item(0) - ans0[0][-1][2].item(0))
    theta0 = theta1
    theta1 = new_theta
    n+=1
theta1*180/pi/0.06

T0 = 15

iteration 0
0.019999999999999997
iteration 1
0.005545358508415789
iteration 2
4.0793524062540665e-06
iteration 3
4.583330451279899e-10


33.94721299580389

In [350]:
global g, d, m, i, v15, P0, T0, w, zt, cd, m3, S, Tv, Tz, v0
g = 9.80665 #ускорение свободного падения
d = 12.7 * 10**-3 #диаметр пули
m = 48.3 * 10**-3 #масса пули
i = 1.0628 #коэффиециен формы
v15 = 820.0 #начальная скорость пули T = 15 c
P0 = 750 #начальное давление
T0 = 5 #начальная температура
w = 0.5 #влажность воздуха
zt = 1.35 * 10**-3 #коэффиециент температуры заряда
cd = 0.0423 #коэффиециент деривации
m3 = 0.1744 #коэффиециент убывания угловой скорости
S = pi*d*d/4
Tv = (T0 + 273.15)/(1 - 3*12.7/(8.0*P0) * w)

def T(y):
    global Tv
    return Tv - 0.0065*y

def p(y):
    global Tv,P0
    return 1.225 * np.power(T(y)/Tv, 4.256) * P0*288.15/(Tv*760.0)

def s_speed(y):
    return 340.924 * np.power(T(y)/288.15, 1/2.0)

def cx(x):
    pa = np.array([0.0525, -0.9476, 8.9342, -9.4610, 0.3207, 4.2980, -1.9382])
    pb = np.array([1.0000, -15.4071, 178.6690, -580.8643, 985.5873, -853.9492, 296.9213])
    pc = np.array([0.0531, 0.9449, 90.5063, 0.1639])
    return np.polyval(pa,x**2)/np.polyval(pb,x**2) + pc[0] / (1+np.exp(-(x-pc[1])*pc[2]))+pc[3]

Tz = T0
v0 = v15*(1+zt*(Tz-15))

def Fs(i,S,p,v,a):
    return i*S*p*v*v/2*cx(v/a)

print('T0 = 5\n')
T0 = 5
theta0 = 0.01
theta1 = 0.1
eps = 10**-15
n = 0 
while abs(theta1 - theta0) > eps:
    print("iteration",n)
    print(abs(theta1 - theta0))
    u0 = np.array([          
        [v0*np.cos(theta1)], #u
        [np.tan(theta1)],   #gamma
        [0.0],                 #y
        [0.0]                  #t
        ])
    ans1 = universal_runge_kutta(f, u0, np.linspace(0,2000,16), butcher, a, b)
    
    u0 = np.array([          
        [v0*np.cos(theta0)],   #u
        [np.tan(theta0)],      #gamma
        [0.0],                 #y
        [0.0]                  #t
        ])
    ans0 = universal_runge_kutta(f, u0, np.linspace(0,2000,16), butcher, a, b)
    new_theta = theta1 - (theta1 - theta0) * ans1[0][-1][2].item(0) / (ans1[0][-1][2].item(0) - ans0[0][-1][2].item(0))
    theta0 = theta1
    theta1 = new_theta
    n+=1
theta1*180/pi/0.06

T0 = 5

iteration 0
0.09000000000000001
iteration 1
0.062124408751425504
iteration 2
1.4863737999346038e-05
iteration 3
6.443842195946736e-10


36.154330978838686

In [351]:
print('T0 = 15\n')
theta0 = 0.0
theta1 = 0.1
eps = 10**-15
n = 0 
while True:
    print("iteration",n)
    theta = (theta0+theta1)/2
    u0 = np.array([          
        [v0*np.cos(theta)], #u
        [np.tan(theta)],   #gamma
        [0.0],                 #y
        [0.0]                  #t
        ])
    ans = universal_runge_kutta(f, u0, np.linspace(0,2000,16), butcher, a, b)[0][-1][2].item(0) 
    if abs(ans) < 10**-5:
        break
    if ans > 0:
        theta1 = theta
    else:
        theta0 = theta
    n+=1
theta*180/pi/0.06

T0 = 15

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20


36.154331402817434